In [1]:
import pandas as pd
import os
from sklearn.model_selection import RepeatedKFold

In [2]:
df = pd.read_excel('SPEAKER.xls')
df=df[df['SUBSET']==' train-clean-100  '].drop(columns=['SUBSET','NAME','MINUTES'])

In [3]:
#Association de l'ID du fichier audio avec son nom du fichier
def associer_nom_fichier(tab):
    liste=os.listdir('LibriSpeech')
    L=[]
    for i in range(len(tab['ID'])):
        name=str(tab['ID'].iloc[i])+'-'
        L.append('LibriSpeech/'+[x for x in liste if x[:len(name)] ==name][0])
    tab['Name']=L
    return(tab)

In [4]:
df = associer_nom_fichier(df)

In [5]:
df

,ID,SEX,Name
3,19,F,LibriSpeech/19-198-0000.flac
8,26,M,LibriSpeech/26-495-0000.flac
9,27,M,LibriSpeech/27-123349-0000.flac
14,32,F,LibriSpeech/32-21625-0000.flac
18,39,F,LibriSpeech/39-121914-0000.flac
...,...,...,...
2451,8747,M,LibriSpeech/8747-293952-0000.flac
2455,8770,M,LibriSpeech/8770-295462-0000.flac
2462,8797,M,LibriSpeech/8797-294123-0000.flac
2469,8838,M,LibriSpeech/8838-298545-0000.flac


In [6]:
#Création d'un dataset train et test équilibrés
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=0)

for train_index, test_index in cv.split(df.drop("SEX", axis = 1)):
    X_train, X_test = df.drop("SEX", axis = 1).iloc[train_index], df.drop("SEX", axis = 1).iloc[test_index]
    y_train, y_test = df[['SEX']].iloc[train_index], df[['SEX']].iloc[test_index]


In [7]:
# Construction d'une nouvelle colonne indiquant si l'observation est dans le train ou le test
X_train['ORIGIN'] = 'TRAIN'
X_test['ORIGIN']= 'TEST' 
X_train['SEX'] = y_train['SEX']
X_test['SEX'] = y_test['SEX']
X_train = X_train.append(X_test)

In [11]:
#Vérification équilibre
X_train.groupby(['ORIGIN', 'SEX']).count()

ID  Name
ORIGIN SEX           
TEST    F    27    27
        M    23    23
TRAIN   F    98    98
        M   103   103

In [13]:
#Exportation
X_train.to_csv("SPEAKERS_TYPE.csv", sep = ';', index = False)